In [1]:
!pip install "tensorflow_hub>=0.6.0"
!pip install "tensorflow>=2.0.0"

In [2]:
import tensorflow as tf
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embeddings = embed([
    "The quick brown fox jumps over the lazy dog.",
    "I am a sentence for which I would like to get its embedding"])

embeddings.shape

TensorShape([2, 512])

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
%cd drive/'My Drive'
!mkdir UniversalEmb
%cd UniversalEmb

/content/drive/My Drive
mkdir: cannot create directory ‘UniversalEmb’: File exists
/content/drive/My Drive/UniversalEmb


In [0]:
!wget https://cogcomp.seas.upenn.edu/Data/QA/QC/train_5500.label
!wget https://cogcomp.seas.upenn.edu/Data/QA/QC/TREC_10.label

In [5]:
!head train_5500.label

DESC:manner How did serfdom develop in and then leave Russia ?
ENTY:cremat What films featured the character Popeye Doyle ?
DESC:manner How can I find a list of celebrities ' real names ?
ENTY:animal What fowl grabs the spotlight after the Chinese Year of the Monkey ?
ABBR:exp What is the full form of .com ?
HUM:ind What contemptible scoundrel stole the cork from my lunch ?
HUM:gr What team did baseball 's St. Louis Browns become ?
HUM:title What is the oldest profession ?
DESC:def What are liver enzymes ?
HUM:ind Name the scar-faced bounty hunter of The Old West .


In [6]:
import pandas as pd
n_example = 5000

labels = [i.split(' ', 1 )[0].split(':')[1] for i in open('train_5500.label', encoding = 'windows-1252').readlines()][:n_example]
texts = [ i.split(' ', 1 )[1][:-1] for i in open('train_5500.label', encoding = 'windows-1252').readlines()][:n_example]
train  = pd.DataFrame({'texts':texts, 'labels': labels})

labels_t = [i.split(' ', 1)[0].split(':')[1] for i in open('TREC_10.label', encoding = 'windows-1252').readlines()]
texts_t = [ i.split(' ', 1)[1][:-1] for i in open('TREC_10.label', encoding = 'windows-1252').readlines()]
test  = pd.DataFrame({'texts':texts_t, 'labels': labels_t})
test.shape

(500, 2)

##MLP with evaluation

In [0]:
tr = {i:j for j, i in enumerate(train.labels.unique().tolist())}
def t(f):
  return tr[f]
train['lb'] = train.labels.apply(t)
test['lb'] = test.labels.apply(t)                  

In [0]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics

In [0]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,1), max_df=0.9, norm='l2', use_idf=True, smooth_idf=True)
tf_res = tf.fit_transform(train.texts)
test_transform = tf.transform(test.texts)
tf_res.shape

(5000, 8007)

In [0]:
import numpy as np
from sklearn.metrics import classification_report
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import accuracy_score
import time

In [0]:
import torch
import numpy as np
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

class WordData(Dataset):
    
    def __init__(self, x_data, y_data):
        
        super().__init__()
        
        self.x_data = x_data
        self.y_data = torch.LongTensor([int(i) for i in y_data])
        
    
    def __len__(self):
        
        return self.x_data.shape[0]
    
    def __getitem__(self, idx):
        
        x = self.x_data[idx].toarray()
        x = torch.FloatTensor(x)
        y = self.y_data[idx]
        
        return x, y


dataset_train = WordData(tf_res, train.lb)
dataset_test = WordData(test_transform, test.lb)

batch_size = 256

train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=batch_size,  shuffle=True)
validation_loader = torch.utils.data.DataLoader(dataset_test, batch_size=batch_size, shuffle=True)

In [0]:
from math import sqrt
class MLP(torch.nn.Module):
    
    def __init__(self, in_features, inner_features, inner_features2, out_features):
        
        super().__init__()
        
        self.linear_1 = torch.nn.Linear(in_features=in_features, out_features=inner_features)
        gl = sqrt(6./(in_features+inner_features))
        self.linear_1.weight.data.uniform_(-gl, gl)
        self.non_linear_function = torch.nn.ReLU()
        self.drop_out = torch.nn.Dropout(0.18)
        self.linear_2 = torch.nn.Linear(in_features=inner_features, out_features=inner_features2)
        gl1 = sqrt(6./(inner_features+inner_features2))
        self.linear_2.weight.data.uniform_(-gl1, gl1)
        self.non_linear_function2 = torch.nn.ReLU()
        self.linear_3 = torch.nn.Linear(in_features=inner_features2, out_features=out_features)
        gl2 = sqrt(6./(inner_features2+out_features))
        self.linear_3.weight.data.uniform_(-gl2, gl2)
  
    def forward(self, x):
        
        # наша привычная схема
        x = self.linear_1(x)
        x = self.non_linear_function(x)
        x = self.drop_out(x)
        x = self.linear_2(x)
        x = self.non_linear_function2(x)
        x = self.linear_3(x)
        
        return x
    

In [0]:
model = MLP(in_features=tf_res.shape[1], inner_features=1524, inner_features2=1024, out_features=len(set(train.lb)))

In [0]:
model

MLP(
  (linear_1): Linear(in_features=8007, out_features=1524, bias=True)
  (non_linear_function): ReLU()
  (drop_out): Dropout(p=0.18, inplace=False)
  (linear_2): Linear(in_features=1524, out_features=1024, bias=True)
  (non_linear_function2): ReLU()
  (linear_3): Linear(in_features=1024, out_features=47, bias=True)
)

In [0]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001,  betas=(0.9, 0.999) )
# aka loss function
criterion = torch.nn.CrossEntropyLoss()

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = model.to(device)

In [0]:
from sklearn.metrics import f1_score
epochs = 15
losses = []
best_test_loss = 10.

test_f1 = []

for n_epoch in range(epochs):
    
    train_losses = []
    test_losses = []
    test_targets = []
    test_pred_class = []
    
    progress_bar = tqdm(total=len(train_loader.dataset), desc='Epoch {}'.format(n_epoch + 1))
    
    model.train()
    
    for x, y in train_loader:
        x = x.to(device)
        y = y.to(device)

        optimizer.zero_grad()

        pred = model(x).squeeze(1)
        loss = criterion(pred, y)
        
        loss.backward()
        
        optimizer.step()
        
        train_losses.append(loss.item())
        losses.append(loss.item())
        
        progress_bar.set_postfix(train_loss = np.mean(losses[-500:]))

        progress_bar.update(x.shape[0])
        
    progress_bar.close()
    
    model.eval()
    
    for x, y in validation_loader:     
        
        x = x.to(device)
        y = y.to(device)
       

        with torch.no_grad():
            pred = model(x).squeeze(1)

            pred = pred.cpu()
            y = y.cpu()

            test_targets.append(y.numpy())
            test_pred_class.append(np.argmax(pred, axis=1))

            loss = criterion(pred, y)

            test_losses.append(loss.item())
        
    mean_test_loss = np.mean(test_losses)

    test_targets = np.concatenate(test_targets).squeeze()
    test_pred_class = np.concatenate(test_pred_class).squeeze()

    f1 = f1_score(test_targets, test_pred_class, average='micro')

    test_f1.append(f1)
    
    print()
    print('Losses: train - {:.7f}, test - {:.7f}'.format(np.mean(train_losses), mean_test_loss))

    print('F1 test - {:.4f}'.format(f1))
        
    # Early stopping:
    if mean_test_loss < best_test_loss:
        best_test_loss = mean_test_loss
    else:
        print('Early stopping')
        break

Epoch 2:   0%|          | 0/5000 [00:00<?, ?it/s]


Losses: train - 3.2278627, test - 2.9266552
F1 test - 0.2240


Epoch 3:   0%|          | 0/5000 [00:00<?, ?it/s]


Losses: train - 2.2749398, test - 2.1534017
F1 test - 0.5760


Epoch 4:   0%|          | 0/5000 [00:00<?, ?it/s]

...


Losses: train - 0.3281097, test - 1.3925610
F1 test - 0.7180


Epoch 7:   0%|          | 0/5000 [00:00<?, ?it/s]


Losses: train - 0.1709112, test - 1.3405072
F1 test - 0.7440


Epoch 7: 100%|██████████| 5000/5000 [00:04<00:00, 1039.69it/s, train_loss=1.14]



Losses: train - 0.0861694, test - 1.3637035
F1 test - 0.7260
Early stopping


In [0]:
for instance in list(tqdm._instances):
    tqdm._decr_instances(instance)

## Embed sentences

In [0]:
!wget https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-v1.zip
!unzip wikitext-103-v1.zip

--2020-02-28 22:24:02--  https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-v1.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.249.174
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.249.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190229076 (181M) [application/zip]
Saving to: ‘wikitext-103-v1.zip’

wikitext-103-v1.zip 100%[===================>] 181.42M  38.0MB/s    in 5.3s    

2020-02-28 22:24:07 (34.5 MB/s) - ‘wikitext-103-v1.zip’ saved [190229076/190229076]



In [0]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def read_wiki(path, n_lines = 100000):
  lines = open(path).readlines()[:n_lines]
  sentences = [ ]
  for i in lines:
    sentences.extend(sent_tokenize(i)) 
  return sentences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
wiki_sentences = read_wiki('wikitext-103/wiki.train.tokens', 150000)

In [0]:
import numpy as np

In [0]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
def embedAndSaveToTxt(sentences, file_name, embed):
  embeded_sentences = embed(sentences)
  print(embeded_sentences.shape)
  np.save(file_name, embeded_sentences.numpy())
  #f = open(file_name, 'w')
  #for i in embeded_sentences.numpy().tolist():
    #for j in i:
      #f.write(str(j)+' ')
    #f.write('\n')
  #f.close()
  print('Saved to {}'.format(file_name))

In [0]:
!ls

Anaconda3-2019.03-Linux-x86_64.sh  train_5500.label  TREC_10.label
augm_emb.npy			   train_emb.npy     wikitext-103


In [0]:
embedAndSaveToTxt(train.texts.tolist(), 'train_emb', embed)
#embedAndSaveToTxt(wiki_sentences, 'augm_emb', embed)

(5000, 512)
Saved to train_emb


##SQuAD

In [7]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json

--2020-04-04 18:27:03--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.110.153, 185.199.109.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  52.5MB/s    in 0.8s    

2020-04-04 18:27:04 (52.5 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]



In [0]:
import json
with open('train-v2.0.json') as json_file:
    data = json.load(json_file)

In [0]:
quest = []
for i in range(442):
  for k in range(1000):
    for h in range(1000):
      try:
        quest.append(data['data'][i]['paragraphs'][h]['qas'][k]['question'])
      except IndexError:
        continue

In [0]:
len(quest()